# Analysis

In [116]:
# import modules
import os           # reading path
import pandas as pd # formating data into dataframe
import datetime as dt     # data rangings


# this ensures that Pandas won't add newline characters 
# to fit the output of a dataframe in the terminal 
# (code is not functionally important)
pd.set_option('display.expand_frame_repr', False)

# Helper Functions

Here is where I define any functions I created

In [117]:
def remove_char_from_columns(df, char):
    columns = list(df.columns)
    for i, column in enumerate(columns):
        if char in column:
            columns[i] = "".join(column.split(char))

    df.columns = columns

## Extract data from csv in '/data' folder

## Testing
This code should be tested for the following cases:
1. Data directory is empty.
2. Folders are in the data directory.
3. Invalid file name formats {only underscores, no underscores, missing file extension, missing name}.

### Todo:
* wrap code in a function or factory pattern so that the account map can generalize to different companies (i.e Chase, Discovery, etc).

In [118]:
# should read path from environment variables
dataPath = r"C:\Users\maxim\Work\financials\data"

# create list of files/folders in data directory
file_names = os.listdir(dataPath)

# account id's are map to the files that contain their transaction data
# NB: id's are given to be small and not likely to change in the future (many reads, rarely any writes)
acc_map = {}

# iterate through the files and match files that are csv's to an id
for file_name in file_names:
    # seperate the file name (i.e "file.txt") from its extension (i.e ["file", ".txt"]) safely (using os.path)
    split_tup = os.path.splitext(file_name)
    # pretext is the name before the "." and the extension (i.e ".txt")
    pretext = split_tup[0]
    # postext is the "." and extension (everything but the file's name)
    posttext = split_tup[1]

    # only consider files matching the extension ".CSV" extension (post call of `upper()` )
    if ".CSV" == posttext.upper():
        # chase accounts follow the pattern: <id>_<date>_<optional dates>.CSV
        # NB: this split depends on the file convention of your company
        id = pretext.split("_")[0]
        # list initialization if new id, otherwise appen filename to exisint id.
        if id in acc_map:
            acc_map[id].append(file_name)
        else:
            acc_map[id] = [file_name]

# checkings and savings accounts are seperated by the "id" in the filename

## Data Frame Creation

Using Pandas, each account will have it's dataframe

In [140]:
df_map = {}
for k, file_names in acc_map.items():
    df_list = []
    for file_name in file_names:
        df_list.append(pd.read_csv(os.path.join(dataPath, file_name)))

    # line of code that merges
    df_map[k] = pd.concat(df_list)
    remove_char_from_columns(df_map[k], " ")
    print(df_map[k])
    

   TransactionDate    PostDate                Description               Category        Type  Amount  Memo
0       08/12/2022  08/12/2022   Payment Thank You-Mobile                    NaN     Payment  201.45   NaN
1       08/10/2022  08/12/2022   TABATA NOODLE RESTAURANT           Food & Drink        Sale  -21.42   NaN
2       08/09/2022  08/10/2022              CHIPOTLE 2964           Food & Drink        Sale  -13.99   NaN
3       08/08/2022  08/09/2022         SQ *2 BROS 9TH AVE           Food & Drink        Sale   -6.00   NaN
4       08/07/2022  08/08/2022       DD DOORDASH GUACTIME           Food & Drink        Sale  -19.22   NaN
5       08/07/2022  08/07/2022          DOORDASH DASHPASS           Food & Drink        Sale   -4.99   NaN
6       08/05/2022  08/07/2022            SNACK* SONG TEA           Food & Drink        Sale   -9.35   NaN
7       08/05/2022  08/07/2022             Steam Purchase          Entertainment        Sale  -11.99   NaN
8       08/05/2022  08/07/2022       

## Analysis

Simple Analysis.

Todo:

Helper function to convert the columns to the correct types (this function might be called earlier)

In [139]:

date_range = (dt.datetime(2002,7,15), dt.datetime.today())

credit_df = df_map['Chase8960']

credit_df['TransactionDate'] = credit_df['TransactionDate'].astype('datetime64[ns]')

credit_expenses = credit_df[credit_df.Amount < 0]
credit_expenses = credit_expenses[credit_expenses.TransactionDate > date_range[0]]

print(credit_expenses[['Amount']].sum())
print(credit_expenses.groupby(['Category']).sum())
print(credit_expenses)
print(credit_expenses.sort_values(['Amount'])[credit_expenses.Category == 'Shopping'])

Amount   -2122.03
dtype: float64
                       Amount  Memo
Category                           
Entertainment          -35.97   0.0
Food & Drink          -749.34   0.0
Groceries             -319.15   0.0
Health & Wellness      -36.88   0.0
Home                   -11.16   0.0
Professional Services  -23.63   0.0
Shopping              -920.35   0.0
Travel                 -25.55   0.0
   TransactionDate    PostDate                Description               Category  Type  Amount  Memo
1       2022-08-10  08/12/2022   TABATA NOODLE RESTAURANT           Food & Drink  Sale  -21.42   NaN
2       2022-08-09  08/10/2022              CHIPOTLE 2964           Food & Drink  Sale  -13.99   NaN
3       2022-08-08  08/09/2022         SQ *2 BROS 9TH AVE           Food & Drink  Sale   -6.00   NaN
4       2022-08-07  08/08/2022       DD DOORDASH GUACTIME           Food & Drink  Sale  -19.22   NaN
5       2022-08-07  08/07/2022          DOORDASH DASHPASS           Food & Drink  Sale   -4.99   NaN
6

C:\Users\maxim\AppData\Local\Temp\ipykernel_29464\4058529355.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(credit_expenses.sort_values(['Amount'])[credit_expenses.Category == 'Shopping'])
